In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [28]:
plt.style.use('fivethirtyeight')

In [29]:
df = pd.read_csv('BBCA.JK.csv')
df

Date    Open    High     Low   Close    Adj Close    Volume
0     2017-01-02  3100.0  3100.0  3100.0  3100.0  2775.055664         0
1     2017-01-03  3100.0  3155.0  3045.0  3155.0  2824.290039  46748000
2     2017-01-04  3170.0  3175.0  3130.0  3140.0  2810.862305  85216500
3     2017-01-05  3140.0  3155.0  3125.0  3135.0  2806.386719  97909000
4     2017-01-06  3090.0  3140.0  3090.0  3120.0  2792.958984  36603000
...          ...     ...     ...     ...     ...          ...       ...
1257  2021-12-24  7300.0  7350.0  7300.0  7300.0  7021.503418  30802100
1258  2021-12-27  7300.0  7350.0  7300.0  7350.0  7069.596191  20722100
1259  2021-12-28  7375.0  7400.0  7325.0  7350.0  7069.596191  39343800
1260  2021-12-29  7350.0  7375.0  7300.0  7300.0  7021.503418  42079500
1261  2021-12-30  7350.0  7350.0  7300.0  7300.0  7021.503418  49954700

[1262 rows x 7 columns]

In [30]:
df.dropna(inplace=True)

In [31]:
def split_data(df, train_size): 
  size = int(len(df) * train_size)
  train, test = df.iloc[0:size], df.iloc[size:len(df)]
  return train, test

train, test = split_data(df['Close'], 0.8) #80% Train

In [32]:
scaler = MinMaxScaler()
train = scaler.fit_transform(train.values.reshape(-1,1))
test = scaler.transform(test.values.reshape(-1,1))

In [33]:
def create_sequences(data, seq_length=60):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)
seq_length=60
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)

In [34]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [35]:
model = Sequential([
    LSTM(128, input_shape=(seq_length, 1), return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2), 
    Dense(32),
    Dense(16),
    Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              metrics=["mae"], 
              loss=tf.keras.losses.Huber())

early_stop = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), shuffle=False, callbacks=[early_stop])

Epoch 1/1000
8/8 [==============================] - 9s 515ms/step - loss: 0.0219 - mae: 0.1805 - val_loss: 0.0366 - val_mae: 0.2586
Epoch 2/1000
8/8 [==============================] - 3s 317ms/step - loss: 0.0120 - mae: 0.1237 - val_loss: 0.0023 - val_mae: 0.0535
Epoch 3/1000
8/8 [==============================] - 3s 331ms/step - loss: 0.0140 - mae: 0.1465 - val_loss: 0.0142 - val_mae: 0.1540
Epoch 4/1000
8/8 [==============================] - 3s 328ms/step - loss: 0.0091 - mae: 0.1100 - val_loss: 0.0065 - val_mae: 0.0972
Epoch 5/1000
8/8 [==============================] - 3s 340ms/step - loss: 0.0060 - mae: 0.0940 - val_loss: 0.0030 - val_mae: 0.0619
Epoch 6/1000
8/8 [==============================] - 3s 418ms/step - loss: 0.0054 - mae: 0.0835 - val_loss: 0.0041 - val_mae: 0.0757
Epoch 7/1000
8/8 [==============================] - 3s 422ms/step - loss: 0.0036 - mae: 0.0682 - val_loss: 0.0017 - val_mae: 0.0444
Epoch 8/1000
8/8 [==============================] - 4s 499ms/step - loss: 0.

8/8 [==============================] - 3s 356ms/step - loss: 0.0020 - mae: 0.0475 - val_loss: 8.0813e-04 - val_mae: 0.0302
Epoch 63/1000
8/8 [==============================] - 3s 371ms/step - loss: 0.0020 - mae: 0.0488 - val_loss: 9.3616e-04 - val_mae: 0.0332
Epoch 64/1000
8/8 [==============================] - 3s 358ms/step - loss: 0.0017 - mae: 0.0430 - val_loss: 8.2189e-04 - val_mae: 0.0306
Epoch 65/1000
8/8 [==============================] - 3s 373ms/step - loss: 0.0017 - mae: 0.0438 - val_loss: 8.5812e-04 - val_mae: 0.0315
Epoch 66/1000
8/8 [==============================] - 3s 373ms/step - loss: 0.0015 - mae: 0.0413 - val_loss: 6.4122e-04 - val_mae: 0.0264
Epoch 67/1000
8/8 [==============================] - 3s 344ms/step - loss: 0.0017 - mae: 0.0446 - val_loss: 8.1098e-04 - val_mae: 0.0304
Epoch 68/1000
8/8 [==============================] - 3s 349ms/step - loss: 0.0014 - mae: 0.0404 - val_loss: 6.8152e-04 - val_mae: 0.0274
Epoch 69/1000
8/8 [==============================] - 3s

8/8 [==============================] - 3s 425ms/step - loss: 0.0012 - mae: 0.0411 - val_loss: 7.9368e-04 - val_mae: 0.0316
Epoch 122/1000
8/8 [==============================] - 4s 428ms/step - loss: 0.0010 - mae: 0.0366 - val_loss: 3.6815e-04 - val_mae: 0.0204
Epoch 123/1000
8/8 [==============================] - 3s 386ms/step - loss: 0.0012 - mae: 0.0401 - val_loss: 6.1878e-04 - val_mae: 0.0270
Epoch 124/1000
8/8 [==============================] - 3s 345ms/step - loss: 0.0010 - mae: 0.0364 - val_loss: 3.5879e-04 - val_mae: 0.0202
Epoch 125/1000
8/8 [==============================] - 3s 351ms/step - loss: 0.0012 - mae: 0.0391 - val_loss: 6.9000e-04 - val_mae: 0.0289
Epoch 126/1000
8/8 [==============================] - 3s 373ms/step - loss: 0.0010 - mae: 0.0357 - val_loss: 3.8355e-04 - val_mae: 0.0206
Epoch 127/1000
8/8 [==============================] - 3s 371ms/step - loss: 0.0011 - mae: 0.0375 - val_loss: 5.7454e-04 - val_mae: 0.0258
Epoch 128/1000
8/8 [=============================

8/8 [==============================] - 4s 478ms/step - loss: 9.6230e-04 - mae: 0.0361 - val_loss: 5.2873e-04 - val_mae: 0.0248
Epoch 180/1000
8/8 [==============================] - 4s 461ms/step - loss: 7.9294e-04 - mae: 0.0322 - val_loss: 3.3669e-04 - val_mae: 0.0197
Epoch 181/1000
8/8 [==============================] - 4s 492ms/step - loss: 9.0459e-04 - mae: 0.0343 - val_loss: 4.9045e-04 - val_mae: 0.0237
Epoch 182/1000
8/8 [==============================] - 4s 465ms/step - loss: 7.4532e-04 - mae: 0.0310 - val_loss: 3.2792e-04 - val_mae: 0.0193
Epoch 183/1000
8/8 [==============================] - 4s 481ms/step - loss: 9.0469e-04 - mae: 0.0343 - val_loss: 4.7258e-04 - val_mae: 0.0232
Epoch 184/1000
8/8 [==============================] - 4s 477ms/step - loss: 7.8661e-04 - mae: 0.0321 - val_loss: 3.3075e-04 - val_mae: 0.0192
Epoch 185/1000
8/8 [==============================] - 4s 489ms/step - loss: 8.0703e-04 - mae: 0.0326 - val_loss: 4.4471e-04 - val_mae: 0.0223
Epoch 186/1000
8/8 [=

Epoch 237/1000
8/8 [==============================] - 4s 482ms/step - loss: 4.4779e-04 - mae: 0.0219 - val_loss: 2.9781e-04 - val_mae: 0.0180
Epoch 238/1000
8/8 [==============================] - 4s 475ms/step - loss: 4.2738e-04 - mae: 0.0215 - val_loss: 2.9026e-04 - val_mae: 0.0178
Epoch 239/1000
8/8 [==============================] - 4s 507ms/step - loss: 4.5693e-04 - mae: 0.0216 - val_loss: 3.3096e-04 - val_mae: 0.0189
Epoch 240/1000
8/8 [==============================] - 4s 477ms/step - loss: 4.3895e-04 - mae: 0.0218 - val_loss: 2.9735e-04 - val_mae: 0.0180
Epoch 241/1000
8/8 [==============================] - 4s 481ms/step - loss: 5.3071e-04 - mae: 0.0242 - val_loss: 3.0412e-04 - val_mae: 0.0181
Epoch 242/1000
8/8 [==============================] - 4s 466ms/step - loss: 4.4965e-04 - mae: 0.0223 - val_loss: 2.9093e-04 - val_mae: 0.0179
Epoch 243/1000
8/8 [==============================] - 4s 512ms/step - loss: 4.5098e-04 - mae: 0.0223 - val_loss: 2.8882e-04 - val_mae: 0.0178
Epoch 

In [ ]:
y_pred = model.predict(X_test)
y_pred = list(np.reshape(y_pred, (1,len(y_pred)))[0])

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

def percentage_accuracy(y_true, y_pred, tolerance=0.1):
    return np.mean(np.abs((y_true - y_pred) / y_true) < tolerance)

accuracy = percentage_accuracy(y_test, y_pred, tolerance=0.1)
print(f"Percentage Accuracy within 10%: {accuracy * 100}%")


In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Actual vs Predicted Stock Prices')
plt.legend()
plt.show()

In [ ]:
# save model
model.save('BBCA.h5')

In [ ]:
# load model
loaded_model = load_model('BBCA.h5')